<a href="https://colab.research.google.com/github/dmejiaho/Parciales/blob/main/Dashboard_punto_1_parcial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

In [ ]:
%%writefile seriefourier.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from ipywidgets import IntSlider
import sympy as sym
import sympy.physics.control as control
sym.init_printing()


# Configurar el nombre de la página y el ícono
st.set_page_config(page_title="Sistema MRA - Circuito RLC", page_icon=":chart_with_upwards_trend:")

# Barra lateral para mostrar los nombres de los involucrados
st.sidebar.title("Involucrados en el Proyecto")
st.sidebar.write("Juan Jeronimo Castaño Rivera")
st.sidebar.write("Daniel Mauricio Mejia Hoyos")
st.sidebar.write("Wilmer Sebastian Perez Cuastumal")

# Título del dashboard
st.title("Análisis de sistemas de segundo orden")

# Explicación del procedimiento
st.markdown("""
### Procedimiento
Se utilizaron los sistemas masa-resorte-amortiguador y circuito RLC para observar como se comportan en general los sistemas de segundo orden (A partir de la forma canónica de estos sistemas se estudia el tipo de amortiguamiento que presenta la respuesta al escalón de cada SLIT).
""")

# Mostrar las funciones de transferencia
st.subheader("Funciones de transferencia")

st.markdown(""
"")
st.latex(r"\text{La función de transferencia general de un sistema MRA, es:}")
st.latex(r"H(s)=\frac{1}{ms^2+cs+k}")
st.latex(r"\text{La función de transferencia general de un circuito RLC, es:}")
st.latex(r"H(s)=\frac{1}{LCs^2+RCs+1}")
st.latex(r"\text{La función de transferencia canónica de un sistema de segundo orden, es:}")
st.latex(r"H(s)=K\frac{w_n^2}{s^2+2 \xi w_n s+w_n^2}")
st.latex(r"\text{donde:}")
st.latex(r"\xi: \text{factor de amortiguamiento}")
st.latex(r"w_n: \text{frecuencia natural}")
st.latex(r"K: \text{ganancia}")
st.latex(r"\text{Según el factor de amortiguamiento, los SLIT de segundo orden se caracterizan como: }")
st.latex(r"0< \xi <1: \text{SLIT subamortiguado}")
st.latex(r"\xi = 1: \text{SLIT con amortiguamiento crítico}")
st.latex(r"\xi > 1: \text{SLIT sobreamortiguado}")
st.latex(r"\xi = 0: \text{SLIT oscilatorio}")
st.latex(r"\xi < 0: \text{SLIT inestable}")
st.latex(r"\text{Para el sistema MRA:}")
st.latex(r"K=\frac{1}{k}")
st.latex(r"w_n^2=\frac{k}{m}")
st.latex(r"\xi = \frac{c}{2\sqrt{k m}}")
st.latex(r"\text{Para el circuito RLC:}")
st.latex(r"K=1")
st.latex(r"w_n^2=\frac{1}{LC}")
st.latex(r"\xi = \frac{R}{2}\sqrt{\frac{C}{L}}")

st.markdown("""
#### Desarrollo
""")
st.latex(r"\text{Si me dan el valor de} \, \xi \, , \text{el programa deberá calcular los valores del sistema MRA y del circuito RLC}")
xi = st.slider("Seleccione el valor del factor de amortiguamiento: ", min_value=-3.0, max_value=5.0, value=0.5)

CC=0.5
LL=0.5
RR=2*xi*np.sqrt(LL/CC)
kk=1/CC
cc=RR
mm=LL

if(xi<0):
  st.write("Para \u03BE menor a cero, el sistema es inestable. En este caso, es imposible plantear sistemas MRA o circuitos RLC inestables. Implicarían parámetros negativos")
  st.write("Sin embargo, se puede realizar el proceso de hallar el diagrama de bode, diagrama de polos y ceros, respuesta escalón y respuesta impulso de manera ilustrativa")
  z=1

elif(xi==0):
  st.write("Para \u03BE igual a cero, el sistema es oscilado.")
  z=0

elif(xi==1):
  st.write("Para \u03BE igual a 1, el sistema es críticamente amortiguado.")
  z=0
elif(xi>1):
  st.write("Para \u03BE mayor a 1, el sistema es sobre-amortiguado.")
  z=0

elif(xi<1 and xi>0):
  st.write("Para \u03BE entre 0 y 1, el sistema es sub-amortiguado.")
  z=0

if (z==0):

  st.write("\n\nLos valores que satisfacen el valor de \u03BE del sistema MRA son:")
  st.write(f"k={kk}N/m")
  st.write(f"c={cc}Ns/m")
  st.write(f"m={mm}kg")

  st.write("\n\nLos valores que satisfacen el valor de \u03BE del sistema MRA son:")
  st.write(f"C={CC}F")
  st.write(f"L={LL}H")
  st.write(f"R={RR}Ω")

st.write(f"\n\nPara nuestro caso, w_n={1/np.sqrt(LL*CC)}")

#RLC = {R: sym.Rational(str(RR)), L: sym.Rational(str(LL)), C: sym.Rational(str(CC))}

s = sym.symbols('s', complex=True)
t, R, L, C = sym.symbols('t R L C', positive=True)
#Entrada de interés en Laplace
X = 1/s # Ajustar según tipo de entrada
H=control.TransferFunction(1,L*C*s**2 + R*C*s + 1,s) #Funcion de transferencia con la API de control
#Hlc = H/(1+H) #para lazo cerrado

RLC = {R: sym.Rational(str(RR)), L: sym.Rational(str(LL)), C: sym.Rational(str(CC))}

Y = H.num * X /H.den#función de transferencia del circuito multiplicada por X(s)
#Función de salida

#Función de salida en Laplace factorizada (fracciones parciales)
Yp = Y.apart(s) # el comando apart factoriza la expresión simbólica

#Modelo en el tiempo en función de RLC
Ye = Yp
#Se recorre cada fracción parcial según factorización para simplificar cálculos
for j in range(len(Ye.args)):
    if j==0:#primer elemento de la solucion
      yt = sym.inverse_laplace_transform(Ye.args[j], s, t)
    else: #siguientes elementos concatenados
      yt+= sym.inverse_laplace_transform(Ye.args[j], s, t)

#Modelo en el tiempo con RLC evaluados
Ye = Yp.subs(RLC)
for j in range(len(Ye.args)):
    if j==0:
      yt = sym.inverse_laplace_transform(Ye.args[j], s, t)
    else:
      yt+= sym.inverse_laplace_transform(Ye.args[j], s, t)

st.markdown("""
#### Diagrama de polos y ceros
""")
#Diagrama de polos y ceros
HRLC=control.TransferFunction(H.num,H.den.subs(RLC),s)
control.pole_zero_plot(HRLC)
st.pyplot(plt.gcf())  # Mostrar la figura
plt.clf()

st.markdown("""
#### Diagrama de Bode
""")
#Diagrama de Bode
control.bode_plot(HRLC)
st.pyplot(plt.gcf())  # Mostrar la figura
plt.clf()

st.markdown("""
#### Respuesta al impulso en el tiempo
""")
#Respuesta al impulso
control.impulse_response_plot(HRLC)
st.pyplot(plt.gcf())  # Mostrar la figura
plt.clf()

st.markdown("""
#### Respuesta al escalón en el tiempo
""")
#Respuesta al impulso
control.step_response_plot(HRLC)
st.pyplot(plt.gcf())  # Mostrar la figura
plt.clf()


Overwriting seriefourier.py


In [ ]:
token = '2maC7FWeV10FqgyYS7bbT7LlG31_VL9v3A7SAuVTyZMyKP12'
#token = '2maC8CGjEgzmGr3l0TWkDHqnKdd_6oU59SUFP74LG7BtMskMV' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [ ]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run seriefourier.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://7172-34-80-52-183.ngrok-free.app
